In [1]:
import os, shutil
from keras import layers
from keras import models
from keras import optimizers
import math

Using TensorFlow backend.


In [2]:
#List of categories that will be considered. The items of this list have the same name as the names of the folders 
#the images are strored in.

list_categories = ['bar','bowling','buffet','casino','concert_hall','fastfood_restaurant','gameroom','gym',
                  'hairsalon','movietheater','restaurant','airport_inside','church_inside','cloister','elevator',
                  'florist','inside_bus','library','locker_room','museum','poolinside','prisoncell','subway',
                   'trainstation','waitingroom']

In [ ]:
# Next we create the file structure to split the images in train, validation and test sets.


#Where the original images are stored.
original_dataset_dir = 'original_images'

#Where we will store the images separated by train, validation and test sets.
base_dir = 'images'

os.mkdir(base_dir)

train_dir = os.path.join(base_dir,'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir,'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir,'test')
os.mkdir(test_dir)

In [6]:
#Inside each of the folders created peveously, we create a folder for each of the categories.

for i in range(len(list_categories)):
    os.mkdir(os.path.join(train_dir,list_categories[i]))
    
for i in range(len(list_categories)):
    os.mkdir(os.path.join(validation_dir,list_categories[i]))
    
for i in range(len(list_categories)):
    os.mkdir(os.path.join(test_dir,list_categories[i]))

In [7]:
#Next we divide the images into train(60%), validation(20%) and test(20%) and copy them to the new file structure.

for i in range(len(list_categories)):
    directory = os.path.join(original_dataset_dir,list_categories[i])
    number_of_files = sum(1 for item in os.listdir(directory) if os.path.isfile(os.path.join(directory, item)))
    train_indice = math.ceil(number_of_files*0.6)
    validation_indice = math.ceil(number_of_files*0.8)
    iterator = 0
    for file in os.listdir(directory):
        if iterator < train_indice:
            src = os.path.join(directory,file)
            dst = os.path.join(train_dir,list_categories[i],file)
            shutil.copyfile(src,dst)
        elif iterator < validation_indice:
            src = os.path.join(directory,file)
            dst = os.path.join(validation_dir,list_categories[i],file)
            shutil.copyfile(src,dst)
        else:
            src = os.path.join(directory,file)
            dst = os.path.join(test_dir,list_categories[i],file)
            shutil.copyfile(src,dst)
        iterator += 1

In [8]:
#Model definition

model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
          
model.add(layers.Flatten())
          
model.add(layers.Dense(512,activation='relu'))

model.add(layers.Dense(25,activation='sigmoid'))

In [9]:
#Model Compilation

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [10]:
#Data genaration for the training and validation steps

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150,150),
        batch_size=20,
        class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
        validation_dir,
        target_size=(150,150),
        batch_size=20,
        class_mode='categorical')

Found 3492 images belonging to 25 classes.
Found 1163 images belonging to 25 classes.


In [11]:
for data_batch, labels_batch in train_generator:
    print('data batch shape: ', data_batch.shape)
    print('labels batch shape: ', labels_batch.shape)
    break

data batch shape:  (20, 150, 150, 3)
labels batch shape:  (20, 25)


In [12]:
#Fitting the model:

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50)

Epoch 1/30
100/100 [==============================] - 43s 426ms/step - loss: 3.0121 - acc: 0.1089 - val_loss: 2.8072 - val_acc: 0.1230
Epoch 2/30
100/100 [==============================] - 47s 473ms/step - loss: 2.8774 - acc: 0.1365 - val_loss: 3.0368 - val_acc: 0.1394
Epoch 3/30
100/100 [==============================] - 47s 475ms/step - loss: 2.7284 - acc: 0.1840 - val_loss: 2.5544 - val_acc: 0.1933
Epoch 4/30
100/100 [==============================] - 47s 466ms/step - loss: 2.6018 - acc: 0.2088 - val_loss: 3.1182 - val_acc: 0.2391
Epoch 5/30
100/100 [==============================] - 49s 486ms/step - loss: 2.5406 - acc: 0.2210 - val_loss: 2.8168 - val_acc: 0.1984
Epoch 6/30
100/100 [==============================] - 50s 497ms/step - loss: 2.4476 - acc: 0.2490 - val_loss: 2.6245 - val_acc: 0.2319
Epoch 7/30
100/100 [==============================] - 51s 512ms/step - loss: 2.4297 - acc: 0.2480 - val_loss: 2.4121 - val_acc: 0.2130
Epoch 8/30
100/100 [==============================] - 5

In [13]:
#Data generation for the test step:

test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (150, 150),
                                            batch_size = 20,
                                            class_mode = 'categorical')

Found 1150 images belonging to 25 classes.


In [14]:
#Evaluate model

model.evaluate_generator(test_set)

[2.6896119117736816, 0.32956522703170776]